In [1]:
!pip install torch
# !pip install torch-1.12.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install timm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
Processing ./torch-1.12.0-cp37-cp37m-manylinux1_x86_64.whl
torch is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [14]:
!python ./compare.py

err: failed to load cuda func: cuCtxGetCurrent


err: Failed to load cuda API library
err: failed to load cuda func: cuCtxGetCurrent
err: failed to load cuda func: cuDeviceGetCount
err: failed to load cuda func: cuGetErrorString
06 09:11:52[mgb] WRN cuda unavailable: unknown cuda error(999) ndev=-1
06 09:11:52 load_serialized_obj_from_url: download to or using cached /home/megstudio/.cache/megengine/serialized/855c16_a9171fa3-eb68-4a74-8d1c-047e37e201ef
abs error: 1.562148077560721e-09
numpy allclose result: True
abs error: 1.5708210288067903e-09
numpy allclose result: True
abs error: 1.582171504921348e-09
numpy allclose result: True
abs error: 1.5761761895660698e-09
numpy allclose result: True
abs error: 1.6250123469063738e-09
numpy allclose result: True
abs error: 1.5736441039138072e-09
numpy allclose result: True
abs error: 1.592561638119605e-09
numpy allclose result: True
abs error: 1.6006234115906182e-09
numpy allclose result: True
abs error: 1.5966652444632246e-09
numpy allclose result: True
abs error: 1.5806290720732363e-09
nu

In [12]:
!python ./compare_modules.py

err: failed to load cuda func: cuCtxGetCurrent


err: Failed to load cuda API library
err: failed to load cuda func: cuDeviceGetCount
err: failed to load cuda func: cuGetErrorString
06 09:05:39[mgb] WRN cuda unavailable: unknown cuda error(999) ndev=-1
Begin test with rtol = 0.001, batch size =8

Begin test Block:
	 with kwards {'dim': 1024, 'drop_path': 0.2, 'layer_scale_init_value': 1e-06}:
		 with shape (8, 1024, 32, 32):
			 with dtype float32:
				Result: True,  100.0000% elements is close enough
 				 which absolute error is  6.287713269969783e-14 and absolute std is 5.03716096611484e-11
				time used: megengine:  1.1543s, torch:  12.2308s
Begin test ConvNeXt:
	 with kwards {'depths': [3, 3, 9, 3], 'dims': [96, 192, 384, 768]}:
		 with shape (8, 3, 32, 32):
			 with dtype float32:
				Result: False,  99.9500% elements is close enough
 				 which absolute error is  4.2248169052072626e-07 and absolute std is 3.2380413017563114e-07
				time used: megengine:  0.4220s, torch:  9.7396s
	 with kwards {'depths': [3, 3, 27, 3], 'dims': 

In [2]:
# megengine
import megengine as mge
import megengine.functional as F
import megengine.module as M
import numpy as np 
import math
import megengine.hub as hub

def drop_path(x, drop_prob: float = 0., training: bool = False, scale_by_keep: bool = True):
    if drop_prob == 0. or not training:
        return x
    keep_prob = mge.tensor(1 - drop_prob, dtype=x.dtype)
    size = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + mge.random.normal(mean=0, std=1, size=size)
    random_tensor = F.floor(random_tensor)  # binarize
    print(random_tensor)
    output = x / keep_prob * random_tensor
    return output

class DropPath(M.Module):

    def __init__(self, drop_prob: float = 0., scale_by_keep: bool = True):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob
        self.scale_by_keep = scale_by_keep

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training, self.scale_by_keep)

    def extra_repr(self):
        return f'drop_prob={round(self.drop_prob,3):0.3f}'


class LayerNorm(M.Module):

    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = mge.Parameter(F.ones((normalized_shape)))
        self.bias = mge.Parameter(F.zeros((normalized_shape)))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.nn.layer_norm(x, self.normalized_shape, True, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdims=True)
            s = F.pow((x - u), 2).mean(1, keepdims=True)
            x = (x - u) / F.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


class Block(M.Module):

    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = M.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = M.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = M.GELU()
        self.pwconv2 = M.Linear(4 * dim, dim)
        self.gamma = mge.Parameter(
            layer_scale_init_value * F.ones((dim))) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else M.Identity()

    def forward(self, x):
        inp = x
        x = self.dwconv(x)
        x = x.transpose(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.transpose(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = inp + self.drop_path(x)
        return x


class ConvNeXt(M.Module):

    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = []# stem and 3 intermediate downsampling conv layers
        stem = M.Sequential(
            M.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = M.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    M.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = []# 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in F.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = M.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = M.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = M.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight *= head_init_scale
        self.head.bias *= (head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (M.Conv2d, M.Linear)):
            M.init.xavier_normal_(m.weight)
            M.init.zeros_(m.bias)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x


@hub.pretrained(
    "https://studio.brainpp.com/api/v1/activities/3/missions/39/files/a9171fa3-eb68-4a74-8d1c-047e37e201ef"
)
def convnext_tiny(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    return model


In [3]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


import torch
import torch.nn as nn
import torch.nn.functional as tF
from timm.models.layers import trunc_normal_, DropPath as torch_DropPath
from timm.models.registry import register_model

class torch_Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = torch_LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = torch_DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class torch_ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf

    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            torch_LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    torch_LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[torch_Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

class torch_LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return tF.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "convnext_small_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
}

@register_model
def torch_convnext_tiny(pretrained=False,in_22k=False, **kwargs):
    model = torch_ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_tiny_22k'] if in_22k else model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

# @register_model
# def convnext_small(pretrained=False,in_22k=False, **kwargs):
#     model = ConvNeXt(depths=[3, 3, 27, 3], dims=[96, 192, 384, 768], **kwargs)
#     if pretrained:
#         url = model_urls['convnext_small_22k'] if in_22k else model_urls['convnext_small_1k']
#         checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
#         model.load_state_dict(checkpoint["model"])
#     return model

# @register_model
# def convnext_base(pretrained=False, in_22k=False, **kwargs):
#     model = ConvNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], **kwargs)
#     if pretrained:
#         url = model_urls['convnext_base_22k'] if in_22k else model_urls['convnext_base_1k']
#         checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
#         model.load_state_dict(checkpoint["model"])
#     return model

# @register_model
# def convnext_large(pretrained=False, in_22k=False, **kwargs):
#     model = ConvNeXt(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
#     if pretrained:
#         url = model_urls['convnext_large_22k'] if in_22k else model_urls['convnext_large_1k']
#         checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
#         model.load_state_dict(checkpoint["model"])
#     return model

# @register_model
# def convnext_xlarge(pretrained=False, in_22k=False, **kwargs):
#     model = ConvNeXt(depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048], **kwargs)
#     if pretrained:
#         assert in_22k, "only ImageNet-22K pre-trained ConvNeXt-XL is available; please set in_22k=True"
#         url = model_urls['convnext_xlarge_22k']
#         checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu")
#         model.load_state_dict(checkpoint["model"])
#     return model


In [6]:
import megengine as mge
import torch
import numpy as np
import sys
import time

mge_model = convnext_tiny(True)
# mge_model.load_state_dict(mge.load('./convnext_tiny_1k_224_ema.pkl'))
torch_model = torch_convnext_tiny(pretrained=True)
torch_time = meg_time = 0.0

def softmax(logits):
    logits = logits - logits.max(-1, keepdims=True)
    exp = np.exp(logits)
    return exp / exp.sum(-1, keepdims=True)

for i in range(15):
    inp = np.random.randn(2, 3, 224, 224)
    mge_inp = mge.tensor(inp, dtype=np.float32)
    torch_inp = torch.tensor(inp, dtype=torch.float32)
    if torch.cuda.is_available():
        torch_inp = torch_inp.cuda()

    st = time.time()
    mge_out = mge_model(mge_inp)
    meg_time += time.time() - st

    st = time.time()
    torch_out = torch_model(torch_inp)
    torch_time += time.time() - st

    if torch.cuda.is_available():
        torch_out = torch_out.detach().cpu().numpy()
    else:
        torch_out = torch_out.detach().numpy()
    mge_out = mge_out.numpy()
    mge_out = softmax(mge_out)
    torch_out = softmax(torch_out)
    print(f"abs error: {np.mean(np.abs(torch_out - mge_out))}")
    print(f"numpy allclose result: {np.allclose(torch_out, mge_out, rtol=1e-3)}")

print(f"meg time: {meg_time}, torch time: {torch_time}")



06 08:56:03 load_serialized_obj_from_url: download to or using cached /home/megstudio/.cache/megengine/serialized/855c16_a9171fa3-eb68-4a74-8d1c-047e37e201ef
100%|███████████████████████████████████████| 114M/114M [00:11<00:00, 10.3MiB/s]
Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /home/megstudio/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


  0%|          | 0.00/109M [00:00<?, ?B/s]

abs error: 1.5822005927645932e-09
numpy allclose result: True
abs error: 1.5312688894653093e-09
numpy allclose result: True
abs error: 1.5734694658320336e-09
numpy allclose result: True
abs error: 1.6321719531475765e-09
numpy allclose result: True
abs error: 1.6188133056260767e-09
numpy allclose result: True
abs error: 1.6298145055770874e-09
numpy allclose result: True
abs error: 1.5592668267672138e-09
numpy allclose result: True
abs error: 1.5797267938211235e-09
numpy allclose result: True
abs error: 1.6069389152661984e-09
numpy allclose result: True
abs error: 1.634762103464027e-09
numpy allclose result: True
abs error: 1.5752157356274665e-09
numpy allclose result: True
abs error: 1.638633007061685e-09
numpy allclose result: True
abs error: 1.5602563685490622e-09
numpy allclose result: True
abs error: 1.6049598317025016e-09
numpy allclose result: True
abs error: 1.5903788286308895e-09
numpy allclose result: True
meg time: 20.72342300415039, torch time: 243.97850036621094


In [15]:
# test alignment between megengine and torch, 
# after testing, I found that some base api are not aligned with the standard(relative error below 1e-3)
# such base api are not aligned:
#   Conv2d
#   GELU
#   GroupNorm
#   LayerNorm (when input is large, I only test the input with shape [8, 512] and  [8, 512, 32, 32])

from functools import partial
import time

GLOBAL_RTOL = 1e-3
BATCH_SIZE = 8
# TEST_SP_DIM = [32, 64, 224, 512, 1024]
TEST_SP_DIM = [32]

DTYPE_MAPPER = {
    # 'float16': (np.float16, torch.float16),
    'float32': (np.float32, torch.float32),
    # 'float64': (np.float64, torch.float64),
}

KWARDS_MAPPER = {
    "Block": [
        {"dim": 1024, "drop_path": 0.2, "layer_scale_init_value":1e-6}
    ],
    "ConvNeXt": [
        {"depths": [3, 3, 9, 3], "dims": [96, 192, 384, 768]}, # tiny
#         {"depths": [3, 3, 27, 3], "dims": [96, 192, 384, 768]}, # small
#         {"depths": [3, 3, 27, 3], "dims": [128, 256, 512, 1024]}, # base
#         {"depths": [3, 3, 27, 3], "dims": [192, 384, 768, 1536]}, # large
    ],
    "LayerNorm": [
        {"normalized_shape": 512, "eps": 1e-6, "data_format": "channels_first"},
        {"normalized_shape": 512, "eps": 1e-6, "data_format": "channels_last"},
    ]
}


CLASS_MAPPER = {
    "Block": (Block, torch_Block),
    "ConvNeXt": (ConvNeXt, torch_ConvNeXt),
    "LayerNorm": (LayerNorm, torch_LayerNorm),
}

# test base api

KWARDS_MAPPER = {
    "Standrd Conv2d": (
        {"in_channels": 512, "out_channels": 1024,
            "kernel_size": 3, "stride": 1, "padding": 1},
        {"in_channels": 256, "out_channels": 512,
            "kernel_size": 7, "stride": 3, "padding": 1},
        {"in_channels": 1024, "out_channels": 1000,
            "kernel_size": 1, "stride": 1, "padding": 0},
    ),
    "Group Conv2d": (
        {"in_channels": 512, "out_channels": 512,
            "kernel_size": 3, "stride": 1, "padding": 1, "groups": 512},
        {"in_channels": 256, "out_channels": 256,
            "kernel_size": 7, "stride": 3, "padding": 1, "groups": 128},
    ),
    "GroupNorm": (
        {"num_groups": 256, "num_channels": 512, "eps": 1e-6, "affine": True},
        {"num_groups": 256, "num_channels": 512, "eps": 1e-6, "affine": False},
    ),
    "LayerNorm": (
        {"normalized_shape": 512, "eps": 1e-6},
    ),
    "Not affine LayerNorm": (
        {"normalized_shape": 512, "eps": 1e-6},
    ),
    "BatchNorm": ( 
        {"num_features": 512, "eps": 1e-6},
        {"num_features": 512, "eps": 1e-5},
    ),
    "MaxPool2d": (
        {"kernel_size": 3, "stride": 2, "padding": 1},
        {"kernel_size": 3, "stride": 1, "padding": 1},
        {"kernel_size": 2, "stride": 2, "padding": 0},
    ),
    "AvgPool2d": (
        {"kernel_size": 3, "stride": 2, "padding": 1},
        {"kernel_size": 3, "stride": 1, "padding": 1},
        {"kernel_size": 2, "stride": 2, "padding": 0},
    )
}


CLASS_MAPPER = {
    "GELU": (M.GELU, nn.GELU),
    "RELU": (M.ReLU, nn.ReLU),
    "LeakyReLU": (M.LeakyReLU, nn.LeakyReLU),
    "SIGMOID": (M.Sigmoid, nn.Sigmoid),
    "MaxPool2d": (M.MaxPool2d, nn.MaxPool2d),
    "AvgPool2d": (M.AvgPool2d, partial(nn.AvgPool2d, count_include_pad=False)),
    "Standrd Conv2d": (M.Conv2d, nn.Conv2d),
    "Group Conv2d": (M.Conv2d, nn.Conv2d),
    "GroupNorm": (M.GroupNorm, nn.GroupNorm),
    "BatchNorm": (M.BatchNorm2d, nn.BatchNorm2d),
    "LayerNorm": (M.LayerNorm, nn.LayerNorm),
    "Not affine LayerNorm": (partial(M.LayerNorm, affine=False), partial(nn.LayerNorm, elementwise_affine=False)),
}


def generate_inputs(shape, dtype='float32'):
    inp = np.random.randn(*shape)
    types = DTYPE_MAPPER[dtype]
    mge_inp = mge.tensor(inp, dtype=types[0])
    torch_inp = torch.tensor(inp, dtype=types[1])
    return mge_inp, torch_inp


def get_atttr_by_name(torch_module, k):
    name_list = k.split('.')
    sub_module = getattr(torch_module, name_list[0])
    if len(name_list) != 1:
        for i in name_list[1:-1]:
            try:
                sub_module = getattr(sub_module, i)
            except:
                sub_module = sub_module[int(i)]
    return torch_module


def convert_state_dict(torch_module, torch_dict):
    mge_dict = {}
    for k, v in torch_dict.items():
        data = v.numpy()
        sub_module = get_atttr_by_name(torch_module, k)
        is_conv = isinstance(sub_module, nn.Conv2d)
        if is_conv:
            groups = sub_module.groups
            is_group = groups > 1
        else:
            is_group = False
        if "weight" in k and is_group:
            out_ch, in_ch, h, w = data.shape
            data = data.reshape(groups, out_ch // groups, in_ch, h, w)
        if "bias" in k and not is_in_string(['norm', 'pwconv'], k):
            if is_conv:
                data = data.reshape(1, -1, 1, 1)
        if "num_batches_tracked" in k:
            continue
        mge_dict[k] = data

    return mge_dict


def is_in_string(targets: list, s: str):
    return any(t in s for t in targets)


def convert_dtype(m):
    pass


def test_func(mge_tensor, torch_tensor):
    mge_out = mge_tensor.numpy()
    if torch.cuda.is_available():
        torch_out = torch_tensor.detach().cpu().numpy()
    else:
        torch_out = torch_tensor.detach().numpy()
    result = np.isclose(mge_out, torch_out, rtol=GLOBAL_RTOL)
    ratio = np.mean(result)
    allclose = np.all(result) > 0
    abs_err = np.mean(np.abs(mge_out - torch_out))
    std_err = np.std(np.abs(mge_out - torch_out))
    return ratio, allclose, abs_err, std_err


def get_channels(kwards):
    for n in ['dim', 'normalized_shape', 'in_channels', 'num_channels', 'normalized_shape', 'num_features']:
        if n in kwards:
            ch = kwards[n]
            if isinstance(ch, list):
                return ch
            return [ch]
    else:
        if 'dims' in kwards:
            return [3]
        return list(np.random.randint(1, 2048, size=[1]))


def main():
    print(f"Begin test with rtol = {GLOBAL_RTOL}, batch size ={BATCH_SIZE}")
    print()
    unalign_list = []
    for k, (mge_class, torch_class) in CLASS_MAPPER.items():
        kwards = KWARDS_MAPPER.get(k, [{}])
        print(f"Begin test {k}:")
        for kw in kwards:
            print(f"\t with kwards {kw}:")
            mge_module = mge_class(**kw)
            mge_module.eval()
            torch_module = torch_class(**kw)
            torch_module.eval()
            channels = get_channels(kw)
            for sp_dim in TEST_SP_DIM:
                input_shape = (BATCH_SIZE, *channels, sp_dim, sp_dim)
                for dtype in DTYPE_MAPPER.keys():
                    mge_inp, torch_inp = generate_inputs(input_shape, dtype)
#                     if "LayerNorm" in k and kw["data_format"] == "channels_last":                    
                    if "LayerNorm" in k:
                        mge_inp = mge_inp.transpose(0, 2, 3, 1)
                        torch_inp = torch_inp.permute(0, 2, 3, 1)
                    print(f"\t\t with shape {mge_inp.shape}:")
                    print(f"\t\t\t with dtype {dtype}:")
                    torch_dict = torch_module.state_dict()
                    mge_dict = convert_state_dict(torch_module, torch_dict)
                    mge_module.load_state_dict(mge_dict)

                    st = time.time()
                    mge_out = mge_module(mge_inp)
                    mge_time = time.time() - st

                    st = time.time()
                    torch_out = torch_module(torch_inp)
                    torch_time = time.time() - st

                    ratio, allclose, abs_err, std_err = test_func(mge_out, torch_out)
                    if not allclose:
                        unalign_list.append(k)
                    print(f"\t\t\t\tResult: {allclose}, {ratio*100 : .4f}% elements is close enough\n \t\t\t\t which absolute error is  {abs_err} and absolute std is {std_err}")
                    print(f"\t\t\t\ttime used: megengine: {mge_time : .4f}s, torch: {torch_time : .4f}s")
    print(f"Test down, unaligned module: {list(set(unalign_list))}")


if __name__ == "__main__":
    a = M.Conv2d(1, 1, 1)
    main()


Begin test with rtol = 0.001, batch size =8

Begin test GELU:
	 with kwards {}:
		 with shape (8, 150, 32, 32):
			 with dtype float32:
				Result: False,  99.9993% elements is close enough
 				 which absolute error is  3.833242701034578e-08 and absolute std is 4.615024806753354e-08
				time used: megengine:  0.0002s, torch:  0.1821s
Begin test RELU:
	 with kwards {}:
		 with shape (8, 468, 32, 32):
			 with dtype float32:
				Result: True,  100.0000% elements is close enough
 				 which absolute error is  0.0 and absolute std is 0.0
				time used: megengine:  0.0001s, torch:  0.1475s
Begin test LeakyReLU:
	 with kwards {}:
		 with shape (8, 1648, 32, 32):
			 with dtype float32:
				Result: True,  100.0000% elements is close enough
 				 which absolute error is  0.0 and absolute std is 0.0
				time used: megengine:  0.0006s, torch:  0.2993s
Begin test SIGMOID:
	 with kwards {}:
		 with shape (8, 1051, 32, 32):
			 with dtype float32:
				Result: True,  100.0000% elements is close eno